# Why solve this project ?
This dataset contains the muliple classes. In this project we are just giving you how to deal with the multiple classes. We are not using all the features here. You will suprise after seen the result. We encourage you to download the dataset and play around it as you have come across far in your Data Science Learning.

After completing this project, you will have the better understanding of how to build a model for multiclass classification. In this project, you will apply the following concepts.

* Train-test split
* Dealing with missing values
* GridSearch CV
* f1_score Evaluation Metrics

In [1]:
#import the packages 
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel      
from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
seg = pd.read_csv(r'C:\Users\tdhoble\Downloads\Predict the Correct Segment.csv',encoding='latin1')
seg.head()

,customer id,channel,Nearest_distance_cosco,Net Sales,Age,children count,Transaction_date,Martial status,Region,Loyality_flag,...,Orders groceries,Orders_cerials,Orders_chokolates,Orders dentals,Orders cosmetics,Orders_ready_eat,Orders_braverage,Orders_frozen,Loyality_amt,segments
0,5102,Email,4,367941,58,4,2017-09-19,0,San Diego,0,...,6,10,1,6,3,7,4,0,335,6
1,3549,Email,33,171305,98,1,2017-02-17,0,Atlanta,0,...,7,8,0,3,5,4,10,8,446,5
2,5885,direct,28,173439,18,2,2017-06-06,0,TampaSt. Petersburg,0,...,5,5,8,9,3,7,5,0,49,6
3,7381,store,40,997988,87,3,2016-09-23,0,Seattle,1,...,5,0,8,1,2,7,7,7,280,3
4,2713,direct,5,898554,31,2,2017-01-20,0,Seattle,0,...,4,5,10,5,10,8,9,0,89,3


In [3]:
categorical = seg.select_dtypes(exclude=['number']).columns
numerical = seg.select_dtypes(include=['number']).columns
print("Numeric Features: \n {}".format(numerical))
print("===="*20)
print("Categorical Features: \n {}".format(categorical))

Numeric Features: 
 Index(['customer id', 'Nearest_distance_cosco', 'Net Sales', 'Age',
       'children count', 'Martial status', 'Loyality_flag', 'coupon amount',
       'Income', 'Orders groceries', 'Orders_cerials', 'Orders_chokolates',
       'Orders dentals', 'Orders cosmetics', 'Orders_ready_eat',
       'Orders_braverage', 'Orders_frozen', 'Loyality_amt', 'segments'],
      dtype='object')
Categorical Features: 
 Index(['channel', 'Transaction_date', 'Region', 'Customer_type', 'Race_cd',
       'payment', 'Brands'],
      dtype='object')


In [4]:
df = pd.concat([pd.get_dummies(seg[categorical]), seg[numerical]], axis=1)
df.head()

,channel_Email,channel_Mobile,channel_direct,channel_store,Transaction_date_2016-01-01,Transaction_date_2016-01-02,Transaction_date_2016-01-03,Transaction_date_2016-01-04,Transaction_date_2016-01-05,Transaction_date_2016-01-06,...,Orders groceries,Orders_cerials,Orders_chokolates,Orders dentals,Orders cosmetics,Orders_ready_eat,Orders_braverage,Orders_frozen,Loyality_amt,segments
0,1,0,0,0,0,0,0,0,0,0,...,6,10,1,6,3,7,4,0,335,6
1,1,0,0,0,0,0,0,0,0,0,...,7,8,0,3,5,4,10,8,446,5
2,0,0,1,0,0,0,0,0,0,0,...,5,5,8,9,3,7,5,0,49,6
3,0,0,0,1,0,0,0,0,0,0,...,5,0,8,1,2,7,7,7,280,3
4,0,0,1,0,0,0,0,0,0,0,...,4,5,10,5,10,8,9,0,89,3


In [5]:
X = df.drop(['customer id','segments'], axis=1)
y = df['segments']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [6]:
classifier = xgb.XGBClassifier(randome_state=2)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')
xgb_cr = classification_report(y_test, y_pred)
print("f1: {},\n classification_report: {}".format(f1, xgb_cr))

f1: 0.18278016502274555,
 classification_report:               precision    recall  f1-score   support

           1       0.20      0.23      0.21       247
           2       0.20      0.10      0.13       288
           3       0.18      0.28      0.22       241
           4       0.15      0.15      0.15       231
           5       0.17      0.13      0.15       251
           6       0.23      0.24      0.24       242

   micro avg       0.19      0.19      0.19      1500
   macro avg       0.19      0.19      0.18      1500
weighted avg       0.19      0.19      0.18      1500



In [7]:
parameters={'learning_rate':[0.1,0.15,0.2,0.25,0.3],
            'max_depth':range(1,3)}
grid_search = GridSearchCV(estimator=classifier, param_grid = parameters, n_jobs=-1, verbose=4)
grid_search.fit(X_train, y_train)
grid_predictions = grid_search.predict(X_test)
grid_f1 = f1_score(y_test, grid_predictions, average='macro')
report = classification_report(y_test, grid_predictions)
print("grid_f1: {}, \nclassification_report: {}".format(grid_f1, report))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\tdhoble\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.7min finished


grid_f1: 0.1778321759637885, 
classification_report:               precision    recall  f1-score   support

           1       0.20      0.23      0.21       247
           2       0.21      0.10      0.13       288
           3       0.17      0.32      0.22       241
           4       0.15      0.16      0.16       231
           5       0.17      0.12      0.14       251
           6       0.21      0.19      0.20       242

   micro avg       0.18      0.18      0.18      1500
   macro avg       0.19      0.19      0.18      1500
weighted avg       0.19      0.18      0.18      1500



In [8]:
model = RandomForestClassifier(random_state=2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')
report = classification_report(y_test, y_pred)

print("f1: {}, \nclassification report: {}".format(f1, report))

C:\Users\tdhoble\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


f1: 0.149282719215076, 
classification report:               precision    recall  f1-score   support

           1       0.17      0.26      0.21       247
           2       0.15      0.12      0.13       288
           3       0.17      0.19      0.18       241
           4       0.11      0.12      0.11       231
           5       0.14      0.11      0.12       251
           6       0.17      0.12      0.14       242

   micro avg       0.15      0.15      0.15      1500
   macro avg       0.15      0.15      0.15      1500
weighted avg       0.15      0.15      0.15      1500

